In [30]:
#Changes made: 
#2/28/21
#Table 1(subreditt's tables) is not going to be returned to Dash to display. 
#Added Planetary Care logo. Add assets folder in notebook folder (where the main script is)
#Remove rolling_mean from sentimental graph

import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table as dt
from dash.dependencies import Input, Output, State
import requests
import textblob
import plotly.express as px
import plotly.graph_objects as go
import datetime

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


def get_pushshift_data(data_type, **kwargs):
    
    base_url = f"https://api.pushshift.io/reddit/search/{data_type}/"
   
    payload = kwargs
   
    request = requests.get(base_url, params=payload)
    return request.json()

def polarity_color(num):
    if num > 0.0:
        return 'blue'
    elif num == 0.0:
        return 'white'
    else:
        return 'red'

#def make_clickable(val):
    # target _blank to open new window
 #   return '<a href="{}">Link</a>'.format(val,val)

def make_clickable(val):
    return '<a href="{}">Link</a>'.format(val)


app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
application = app.server

#app.layout = html.Div(html.Img(src=app.get_asset_url('planetary_care_logo.jpg')))

app.layout = html.Div([
    html.Img(src=app.get_asset_url('planetary_care_logo.jpg')),
    html.H1("RegAg Analysis Portal", style={'color': 'blue'}),
    
    html.Div([
    html.H2("Subreddit comments", style={'color': 'blue'}),
    html.Div('''
        Enter a phrase:'''),
    dcc.Input(id='input_1_state', value='regenerative agriculture', type='text'),
   
    html.Div("Choose the number of days for which the data will be collected for?"),
    dcc.Dropdown(id='dropdown_1', options=[{'label': 'A Week Ago', 'value': '7d'}, {'label': ' 2 Weeks Ago', 'value': '14d'},{'label':' A Year Ago','value':'1y'}], 
                 value = '7d'),   
    html.Button(id='submit-button1', children='Submit'),
        
    ],style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw', 'width':'5'}),
    
    html.Div([
    #html.Div(id="table1"),
    dcc.Graph(id='graph1'),], style={'display': 'inline-block', 'vertical-align': 'top','margin-left': '3vw', 
                                     'margin-top': '3vw', 'width':'5'}), 

############################################################################################################
    html.Div([
    html.H2("Sentimental Analysis", style={'color': 'blue'}),
     html.Div('''
        Enter the phrase:'''),
    dcc.Input(id='input_2_state', value='regenerative agriculture', type='text'),
   
             
    html.Div("Choose the number of days for which the data will be collected for?"),
    dcc.Dropdown(id='dropdown_2', options=[{'label': 'A Week Ago', 'value': '7d'}, {'label': ' 2 Weeks Ago', 'value': '14d'},{'label':' A Year Ago','value':'1y'}], 
                 value = '7d'),   
    html.Button(id='submit-button2', children='Submit'),],style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw', 'width':'5'}),
    
    html.Div([
        html.Div(id="table2"),
        dcc.Graph(id='graph2'),], style={'display': 'inline-block', 'vertical-align': 'top','margin-left': '3vw', 'margin-top': '3vw', 'width':'5'}) 

#########################

])

#], style={'display': 'inline-block', 'vertical-align': 'top', 'margin-left': '3vw', 'margin-top': '3vw', 'margin-bottom':'3vw', 'margin-right':'3vw'} ) 



@app.callback(#Output('table1','children'),
              Output('graph1', 'figure'),
              Output('table2','children'),
              Output('graph2', 'figure'),
              
              [Input('input_1_state', 'value')],
              [Input('dropdown_1', 'value')],
              
              [Input('input_2_state', 'value')],
              [Input('dropdown_2', 'value')],
              [State('submit-button1','n_clicks')],
              [State('submit-button2','n_clicks')]
              )

def get_new_data(input_1_state_value,dropdown_1_value,input_2_state_value, dropdown_2_value,submit_button1_n_clicks, submit_button2_n_clicks):
   
    json_data=get_pushshift_data(data_type="comment", q=input_1_state_value, after=dropdown_1_value, aggs="subreddit", size=1000)
    #print("I am in json_data")
    data=json_data.get("data")
    df = pd.DataFrame(data)
    df['count'] = 1
    min_df = df[['subreddit', 'count']]
    grouped_df = min_df.groupby(['subreddit']).sum()
    grouped_df = grouped_df.sort_values(by=['count'], ascending=False)[0:10]
    grouped_df = grouped_df.reset_index()
    grouped_df.head() #top 5 subreddit where the phrase appears most frequently
    columns =  [{"name": i, "id": i,} for i in (grouped_df.columns)]
    
    #################
   

    fig1= px.bar(grouped_df,              # our dataframe
       x="subreddit",         # x will be the 'key' column of the dataframe
       y="count",   # y will be the 'doc_count' column of the dataframe
       title=f"Subreddits with " + input_1_state_value + " activity",
       labels={"count": "# comments","subreddit": "Subreddits"}, # the axis names
       color_discrete_sequence=["blue"], # the colors used
       height=500,
       width=800)
    #################
    
   
    
    json_data2=get_pushshift_data(data_type="comment", q=input_2_state_value, after=dropdown_2_value,size=10, sort_type="score", sort="desc")
    #print("I am in json_data2")
    data2=json_data2.get("data")
    #df2 = pd.DataFrame(data2)
    # define columns of interest
    columns_of_interest = ["author", "body", "created_utc", "score", "permalink"]

    # transform the response into a dataframe with relevant columns
    df2= pd.DataFrame.from_records(data2)[columns_of_interest]

    columns2 =  [{"name": i, "id": i,} for i in (df2.columns)]
    
    #### create a column with sentiment polarity
    df2["sentiment_polarity"] = df2.apply(lambda row: textblob.TextBlob(row["body"]).sentiment.polarity, axis=1)
    
    # append the string to all the permalink entries so that there's a link to the comment
    df2['permalink'] = "https://reddit.com" + df2['permalink'].astype(str)
    
    #df2['permalink'] = df2.permalink.apply(make_clickable)

    # style the last column to be clickable and print
    #df2.style.format({'permalink': make_clickable})
    
    #### create a column with sentiment subjectivity
    df2["sentiment_subjectivity"] = df2.apply(lambda row: textblob.TextBlob(row["body"]).sentiment.subjectivity, axis=1)

    ### create a column with 'positive' or 'negative' depending on sentiment_polarity
    df2["sentiment"] = df2.apply(lambda row: "positive" if row["sentiment_polarity"] >= 0 else "negative", axis=1)
    df2['polarity_color']=df2.apply(lambda x: polarity_color(x['sentiment_polarity']), axis=1 )
    ### create a column with a text preview that shows the first 50 characters
    df2["preview"] = df2["body"].str[0:50]

    ### take the created_utc parameter and tranform it into a datetime column
    df2["date"] = pd.to_datetime(df2['created_utc'],unit='s')
    
    df2.sort_values(by='date', inplace=True)
    df2.index = pd.to_datetime(df2['date'])
    
    df2['mean'] = df2['sentiment_polarity'].expanding().mean()
    #df2['rolling'] = df2['sentiment_polarity'].rolling('4h').mean()
    df2['scaled_polarity']=abs(1000*df2.sentiment_polarity)
    
    #data3=df2.to_dict('rows')
    df3=df2[['author','preview','score','permalink']]
    #columns2 =  [{"name": i, "id": i,} for i in (df3.columns)]
    columns2= [dict(name='author', id='author', type='text'),
               dict(name='preview', id='preview', type='text'),
               dict(name='score', id='score', type='text'),
               dict(name='permalink', id='permalink', state='active')]
    
    date_max=df2.date.max()
    max_str_date=date_max.strftime("%m/%d/%Y")
    date_min=df2.date.min()
    min_str_date=date_min.strftime("%m/%d/%Y")
    
    #################
   
    fig2 = go.Figure()

    # Add traces
    fig2.add_trace(go.Scatter(x=df2.date,
                                y=df2.sentiment_polarity,
                                mode='markers',
                                marker=dict(
                                            color=df2.polarity_color,
                                            size=df2.scaled_polarity,
                                            sizemode='area',
                                            sizemin=2),
                                 #hovertext=df2.preview))
                                 text=df2.preview,showlegend=False))

    #fig2.add_trace(go.Scatter(x=df2.date, y=df2['rolling'],line = dict(color='green', width=5, dash='dot'),name='Rolling Mean'))
    fig2.add_trace(go.Scatter(x=df2.date, y=df2['mean'],line = dict(color='yellow', width=5, dash='dot'),name='Expanding Mean'))

    #fig.update_layout(title=f'Comment Sentiment in the World News for the past ' + data_days[0] + ' days')
    fig2.update_layout(title=f'Comments sentiments in the World News from ' + min_str_date + ' to ' + max_str_date + ' for the phrase ' + input_2_state_value)
    fig2.update_xaxes(title='Date').update_yaxes(title='Comment Polarity')
  
    #fig2.update_layout(width=800,height=500)

    ################
    #return dt.DataTable(data=grouped_df.to_dict('rows'), columns=columns), fig1, dt.DataTable(data=df3.to_dict('rows'), columns=columns2), fig2
    return fig1, dt.DataTable(data=df3.to_dict('rows'), columns=columns2), fig2

 
        
if __name__ == '__main__':
    application.run(debug=False, port=8080)        

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8080/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Apr/2021 13:11:59] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 13:12:00] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Apr/2021 13:12:00] "GET /_dash-dependencies HTTP/1.1" 200 -


regenerative agriculture
7d


127.0.0.1 - - [05/Apr/2021 13:12:01] "POST /_dash-update-component HTTP/1.1" 200 -
